# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [264]:
# import libraries

import numpy as np
import pandas as pd
import nltk
import sqlite3
import sqlalchemy
import sys
import io
import warnings

from typing import List

from sqlalchemy import create_engine


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, fbeta_score

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [106]:
# Suppress the specific UserWarning(tokenizer in NLP vectorizer)
#warnings.filterwarnings("ignore", message="The parameter 'token_pattern' will not be used since 'tokenizer' is not None'")
warnings.simplefilter(action="ignore", category=FutureWarning)

In [108]:
# Connect to known databases
conn_chrismoetable = sqlite3.connect('chrismoetable.db')
conn_disasterresponse = sqlite3.connect('DisasterResponse.db')
conn_insertdb = sqlite3.connect('InsertDatabaseName.db')

def list_tables(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return connection.execute(query).fetchall()

# List tables for each database
chrismoetable_tables = list_tables(conn_chrismoetable)
disasterresponse_tables = list_tables(conn_disasterresponse)
insertdatabase_tables = list_tables(conn_insertdb)

# Close the connections
conn_chrismoetable.close()
conn_disasterresponse.close()
conn_insertdb.close()

# Output the results
print(chrismoetable_tables)
print(disasterresponse_tables)
print(insertdatabase_tables)

[]
[('Message',)]
[('process_data_table',), ('final',), ('chrismo',)]


In [110]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('chrismo', con=engine)

In [112]:
df.head(1)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
#reviewing data's completeness
df.isna().sum().sum()

0

In [116]:
df.nunique()

id                        26180
message                   26177
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_related               2
floods  

In [118]:
df.head()

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,direct,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
df['genre'].value_counts()

genre
news      13054
direct    10766
social     2396
Name: count, dtype: int64

In [122]:
X = df['message']
Y = df.iloc[:,4:]

In [124]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [126]:
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [128]:
print(X.shape)
print(Y.shape)

(26216,)
(26216, 34)


In [130]:
print(X.isna().sum().sum())
print(Y.isna().sum().sum())

0
0


### 2. Write a tokenization function to process your text data

In [133]:
def tokenize(text: str)-> List[str]:
    """
    Tokenize and process the input text.

    This function performs tokenization, stopword removal, part o speech tagging, and lemmatization on the input text.

    Args:
        text (str): The input text to be processed.

    Returns:
        list: A list of processed tokens.
    """
    lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    STOPWORDS = stopwords.words("english")
    word_tokens = [word for word in word_tokens if word not in STOPWORDS]   #stop word removal
    
    tokens = []
    for word_token in word_tokens:
        token = lemmatizer.lemmatize(word_token.lower().strip(), pos='v')        #pos tagging
        tokens.append(token)
        
    return tokens

In [135]:
#confirming function

tokenize?

Signature: tokenize(text: str) -> List[str]
Docstring:
Tokenize and process the input text.

This function performs tokenization, stopword removal, part o speech tagging, and lemmatization on the input text.

Args:
    text (str): The input text to be processed.

Returns:
    list: A list of processed tokens.
File:      /var/folders/y1/2cln1s193dgc6xy3yzry761r0000gn/T/ipykernel_6191/190188781.py
Type:      function

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [138]:
print(X.shape)
print(Y.shape)

(26216,)
(26216, 34)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size = 0.25)

In [89]:
classifiers = [
                RandomForestClassifier
                    (n_estimators=100, 
                     max_depth=10, 
                     min_samples_split=2, 
                     min_samples_leaf=1, 
                     max_features='sqrt', 
                     n_jobs=-1, 
                     random_state=42),
    
                SVC
                    (C=1.0, 
                    kernel='rbf', 
                    random_state=42, 
                    gamma='scale', 
                    probability=True),
    
                DecisionTreeClassifier
                    (max_depth=10, 
                    min_samples_split=2, 
                    min_samples_leaf=1, 
                    random_state=42, 
                    max_features='sqrt'),
    
                CatBoostClassifier
                    (iterations=100, 
                    depth=6, 
                    learning_rate=0.1, 
                    random_state=42, 
                    l2_leaf_reg=3, 
                    bootstrap_type='Bernoulli', 
                    subsample=0.8, 
                    eval_metric='AUC', 
                    thread_count=-1),
    
                XGBClassifier
                    (n_estimators=100, 
                    max_depth=3, 
                    learning_rate=0.1, 
                    random_state=42, 
                    subsample=0.8, 
                    colsample_bytree=0.8, 
                    objective='binary:logistic', 
                    n_jobs=-1)
]

In [161]:
pipeline = Pipeline([
            ('features', FeatureUnion([
                ('text_pipeline', TfidfVectorizer(tokenizer=tokenize))
            ])),
            ('classifier', MultiOutputClassifier(RandomForestClassifier
                    (n_estimators=100, 
                     max_depth=10, 
                     min_samples_split=2, 
                     min_samples_leaf=1, 
                     max_features='sqrt', 
                     n_jobs=-1, 
                     random_state=42)))
            ])

In [163]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 TfidfVectorizer(tokenizer=<function tokenize at 0x32b11bce0>))])),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                                        n_jobs=-1,
                                                                        random_state=42)))])

In [169]:
y_test_pred = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(test_accuracy)

0.4411046689044858


In [177]:
pipeline = Pipeline([
            ('features', FeatureUnion([
                ('text_pipeline', TfidfVectorizer(tokenizer=tokenize))
            ])),
            ('classifier', MultiOutputClassifier(SVC
                    (C=1.0, 
                    kernel='rbf', 
                    random_state=42, 
                    gamma='scale', 
                    probability=True)))
            ])

In [ ]:
y_test_pred = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(pipeline.estimator.__name__)
print(test_accuracy)

In [268]:
random_forest = RandomForestClassifier(random_state=42)
xg = XGBClassifier(random_state=42)
dec_tree = DecisionTreeClassifier(random_state=42)
kn = KNeighborsClassifier()
naive = MultinomialNB()

In [270]:
models = [kn, naive, random_forest, xg, dec_tree]

In [272]:
def train_predict():
    
    for model in models:
        pipeline = Pipeline([
                ('features', FeatureUnion([
                    ('text_pipeline', TfidfVectorizer(tokenizer=tokenize))
                ])),
                ('classifier', MultiOutputClassifier(model))
                ])
        print(model.__class__.__name__)
        pipeline.fit(X_train, y_train)
        test_accuracy = accuracy_score(y_test, pipeline.predict(X_test))
        print(f'Test Accuracy = {test_accuracy}')
        train_accuracy = accuracy_score(y_train, pipeline.predict(X_train))
        print(f'Train Accuracy = {train_accuracy}')
        print('----------------------------------------------------------------------------------------')
        

In [274]:
train_predict()

KNeighborsClassifier
Test Accuracy = 0.42523649679584985
Train Accuracy = 0.4837249516834503
----------------------------------------------------------------------------------------
MultinomialNB
Test Accuracy = 0.3642050656087885
Train Accuracy = 0.3977723527616723
----------------------------------------------------------------------------------------
RandomForestClassifier
Test Accuracy = 0.4104363747329875
Train Accuracy = 0.9971518665446037
----------------------------------------------------------------------------------------
XGBClassifier
Test Accuracy = 0.4247787610619469
Train Accuracy = 0.6004984233546944
----------------------------------------------------------------------------------------
DecisionTreeClassifier
Test Accuracy = 0.2551113823619164
Train Accuracy = 0.9979656189604312
----------------------------------------------------------------------------------------


In [46]:
results = {}

# Training + evaluating each base classifier
for clf in classifiers:
    clf_name = clf.__class__.__name__
    pipeline.set_params(classifier__estimator=clf)
    
    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    
    # Get training accuracy
    y_train_pred = pipeline.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    
    # Get test accuracy
    y_test_pred = pipeline.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    results[clf_name] = {
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy
    }

# Print results
for clf_name, scores in results.items():
    print(f"{clf_name}:")
    print(f"  Training Accuracy: {scores['train_accuracy']:.3f}")
    print(f"  Test Accuracy: {scores['test_accuracy']:.3f}")
    print()

KeyboardInterrupt: 

In [248]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 TfidfVectorizer(tokenizer=<function tokenize at 0x314fa6520>))])),
                ('classifier',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=None,
                                                               colsample_bynode=None,
                                                               colsample_bytree=0.8,
                                                               device=None,
                                                               early_stopping_rounds=None,
                                                               en...
                                                               gamma=None,
                                                               grow_policy=None,
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=0.1,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=3,
                                                               max_leaves=None,
                                                               min_child_weight=1,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               random_state=42, ...)))])

In [249]:
y_test_pred = pipeline.predict(X_test)

In [250]:
y_train_pred = pipeline.predict(X_train)

In [251]:
accuracy_score(y_test, y_test_pred)

0.4394263045468416

In [252]:
accuracy_score(y_train, y_train_pred)

0.4665852914250839

In [ ]:
def compare_models(x: list):

In [ ]:
def capture_classification_report(y_true, y_pred):
    # Redirect stdout to capture the print output
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    try:
        print(classification_report(y_true, y_pred))
    finally:
        sys.stdout = old_stdout
    return buffer.getvalue()

# Iterate through each column in the target dataframe Y
for column in Y.columns:
    report = capture_classification_report(y_test[column], y_pred[:, Y.columns.get_loc(column)])
    print(f"Category: {column}\n")
    print(report)
    print("\n" + "="*80 + "\n")

In [108]:
# List of models to evaluate, including XGBoost
models = {
    'Random Forest': RandomForestClassifier(n_estimators=400, max_depth=32, min_samples_split=10, n_jobs=-1),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
}

In [112]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
        ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=400, max_depth=32, min_samples_split=10, n_jobs=-1)))
    ])

In [120]:
pipeline.fit(X_train, Y_train)

/Users/chrismo/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x32d8a4e00>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())]))])),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=32,
                                                                        min_samples_split=10,
                                                                        n_estimators=400,
                                                                        n_jobs=-1)))])

In [140]:
print(X.shape)
print(Y.shape)

(26386,)
(26386, 36)


In [134]:
y_predictions = pipeline.predict(X_test)

In [135]:
y_predictions_df = pd.DataFrame(y_predictions, columns = y_test.columns)
y_predictions_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [136]:
for column in y_test.columns:
    print('....................................................\n')
    print('FEATURE: {}\n'.format(column))
    print(classification_report(y_test[column],y_predictions_df[column]))

....................................................

FEATURE: related



ValueError: Found input variables with inconsistent numbers of samples: [6597, 5278]

In [124]:
def model_report(y_test, y_pred):
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [126]:
model_report(y_test, y_pred)

Feature 1: related


ValueError: Found input variables with inconsistent numbers of samples: [6597, 5278]

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Iterate over each model
for name, model in models.items():
    # Create a pipeline for each model
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ]))
        ])),
        ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators=400, max_depth=32, min_samples_split=10, n_jobs=-1)))
    ])
    
    # Fit the model on the training data
    pipeline.fit(X_train, Y_train)
    
    # Predict on the test data
    y_pred = pipeline.predict(X_test)
    
    # Output classification report for each model
    print(f"Classification Report for {name}:\n")
    print(classification_report(Y_test, y_pred))
    print("\n" + "="*60 + "\n")

In [ ]:
acc = accuracy_score(y_test, y_pred)
print(acc)

accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def capture_classification_report(y_true, y_pred):
    # Redirect stdout to capture the print output
    old_stdout = sys.stdout
    sys.stdout = buffer = io.StringIO()
    try:
        print(classification_report(y_true, y_pred))
    finally:
        sys.stdout = old_stdout
    return buffer.getvalue()

# Iterate through each column in the target dataframe Y
for column in Y.columns:
    report = capture_classification_report(y_test[column], y_pred[:, Y.columns.get_loc(column)])
    print(f"Category: {column}\n")
    print(report)
    print("\n" + "="*80 + "\n")

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
                'clf__estimator__n_estimators': [100, 200],
                'clf__estimator__max_depth': [3, 6, 10],
                'clf__estimator__learning_rate': [0.01, 0.1]
    
              }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=1, cv=3)

In [ ]:
print(cv)

In [ ]:
# Fit the grid search
cv.fit(X_train, y_train)

In [ ]:
# Best parameters
best_params = cv.best_params_
#preditcitng using best estimator 
print("Best parameters found: ", best_params)

In [ ]:
y_pred_grid = cv.best_estimator_.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
accuracy = accuracy_score(y_test, y_pred_grid)
precision = precision_score(y_test, y_pred_grid, average='micro')
recall = recall_score(y_test, y_pred_grid, average='micro')


In [ ]:
print(f"Accuracy: {accuracy}")

In [ ]:
print(f"Precision: {precision}")

In [ ]:
print(f"Recall: {recall}")

In [ ]:
print("\nClassification Report:\n", classification_report(y_test, y_pred_grid))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.